<a href="https://colab.research.google.com/github/lisaadnr/ekstraksi-llama/blob/main/Extracted_Entity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Membersihkan 3 File Drive

In [ ]:
import re
import os

# Tentukan folder input dan output
input_folder_path = '/content/sample_data/input2'  # Ganti dengan folder yang sesuai
output_file_path = '/content/sample_data/output/output_text.txt'  # Ganti dengan lokasi output yang diinginkan

def extract_named_entities(text):
    sentences = re.split(r'(?<=\.)\s*', text)  # Split by period followed by spaces
    seen_entities = set()
    result_entities = []

    for sentence in sentences:
        # Modified regex to match multiple words in one entity
        entities_in_sentence = re.findall(r'([^\s/]+(?:\s+[^\s/]+)*)/[^O\s]+', sentence)  # Match words with labels other than /O
        for entity in entities_in_sentence:
            if entity not in seen_entities:
                result_entities.append(entity.strip())  # Add the entity to the result
                seen_entities.add(entity)  # Mark the entity as seen

    return result_entities

# Menulis hasil ke file output
with open(output_file_path, 'w') as output_file:
    # Menulis header untuk file output
    output_file.write("Entity\n")

    # Mengolah setiap file dalam folder input
    for filename in os.listdir(input_folder_path):
        if filename.endswith('.txt'):  # Hanya memproses file .txt
            file_path = os.path.join(input_folder_path, filename)

            # Membaca file
            with open(file_path, 'r') as file:
                text = file.read()

            # Ekstraksi entitas bernama dari teks
            named_entities = extract_named_entities(text)

            # Menyimpan hasil ekstraksi ke file output
            for entity in named_entities:
                output_file.write(f"{entity}\n")

# Konfirmasi bahwa file output telah disimpan
print(f"Hasil ekstraksi disimpan di: {output_file_path}")


Hasil ekstraksi disimpan di: /content/sample_data/output/output_text.txt


### Membersihkan O di depan dan baris hanya spasi

In [ ]:
import re

# Tentukan path file input dan output
input_file_path = '/content/sample_data/output/output_text.txt'  # File input yang berisi entitas dengan 'O'
output_file_path = '/content/sample_data/output/output_text_cleaned.txt'  # File output yang sudah dibersihkan

def clean_entities(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    cleaned_lines = []

    for line in lines:
        line = line.strip()  # Hapus spasi di depan dan belakang

        if not line:  # Jika baris kosong (hanya berisi spasi), lewati
            continue

        # Hapus 'O' di depan entitas jika ada
        line = re.sub(r'^O\s+', '', line)  # Menghapus 'O' dan spasi setelahnya di awal entitas

        # Tambahkan entitas yang sudah dibersihkan ke dalam list
        cleaned_lines.append(line)

    # Menyimpan hasil ke file output
    with open(output_file, 'w') as outfile:
        for cleaned_line in cleaned_lines:
            outfile.write(f"{cleaned_line}\n")

# Proses file input dan hasilkan output yang sudah dibersihkan
clean_entities(input_file_path, output_file_path)

# Konfirmasi bahwa file output telah disimpan
print(f"File hasil pembersihan disimpan di: {output_file_path}")


File hasil pembersihan disimpan di: /content/sample_data/output/output_text_cleaned.txt


## Ekstraksi File JSON GPT-4

In [ ]:
import json
import os
import re

# Path ke folder yang berisi file JSON
folder_path = '/content/sample_data/input'  # Ganti dengan folder input Anda

# Fungsi untuk mengekstrak entitas menggunakan regex
def extract_named_entities(labels):
    # Mengambil string label dan memecahnya berdasarkan spasi
    # Jika labels adalah list of strings, kita gabungkan menjadi satu string
    if isinstance(labels, list):
        label_string = ' '.join(labels)
    else:
        label_string = labels  # Jika sudah berupa string, pakai langsung

    # Regex untuk mengekstrak entitas dengan label selain '/O'
    entities = re.findall(r'([^\s/]+(?:\s+[^\s/]+)*)/[^O\s]+', label_string)

    return entities

# Membaca setiap file JSON di folder dan memprosesnya
output = []

# Menelusuri semua file di dalam folder
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):  # Pastikan hanya file JSON yang diproses
        file_path = os.path.join(folder_path, filename)

        # Membaca data dari file JSON
        with open(file_path, 'r', encoding='utf-8') as f:
            try:
                data = json.load(f)

                # Memproses setiap chapter dan verse dalam file JSON
                for chapter, verses in data.items():
                    for verse_id, verse_data in verses.items():
                        # Ekstraksi entitas dari label
                        # Pastikan kita mengakses dengan benar, baik itu list atau string
                        labels = verse_data.get('labels', {}).get('id', '')  # Ambil id dari labels, jika ada

                        # Jika labels tidak ada atau tidak valid, lanjutkan
                        if not labels:
                            continue

                        # Ekstraksi entitas
                        entities = extract_named_entities(labels)

                        # Menambahkan entitas ke output dalam format tuple untuk memudahkan pengurutan
                        for entity in entities:
                            output.append((int(verse_data['chapterid']), int(verse_data['verse_id']), entity))
            except json.JSONDecodeError:
                print(f"File {filename} tidak dapat dibaca sebagai JSON.")

# Mengurutkan output berdasarkan chapterid dan verse_id
output.sort(key=lambda x: (x[0], x[1]))  # Urutkan berdasarkan chapterid dan verse_id

# Menulis hasil output ke file TSV
output_file = '/content/sample_data/output/output_gpt4.tsv'
with open(output_file, 'w', encoding='utf-8') as f:
    # Menulis header
    f.write("chapter_id\tverse_id\tentitas\n")

    # Menulis data entitas yang sudah diurutkan
    for line in output:
        f.write(f"{line[0]}\t{line[1]}\t{line[2]}\n")

print(f"Output telah disimpan dalam file: {output_file}")


Output telah disimpan dalam file: /content/sample_data/output/output_gpt4.tsv


## Cleaning huruf O

In [ ]:
import csv

# Path ke file input dan output
input_file_path = '/content/sample_data/output/output_gpt4.tsv'  # File input yang berisi data yang ingin diproses
output_file_path = '/content/sample_data/output/output_cleaned_gpt4.tsv'  # File output yang sudah dibersihkan

def clean_entities(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        reader = csv.reader(infile, delimiter='\t')

        # Menulis hasil ke file output
        with open(output_file, 'w', encoding='utf-8', newline='') as outfile:
            writer = csv.writer(outfile, delimiter='\t')

            # Menulis header ke file output
            header = next(reader)  # Membaca header dan menulis ke file output
            writer.writerow(header)

            for row in reader:
                chapter_id = row[0]
                verse_id = row[1]
                entity = row[2].strip()  # Menghapus spasi di depan dan belakang entitas

                # Hapus 'O' di depan entitas jika ada
                if entity.startswith('O '):
                    entity = entity[2:].strip()  # Menghapus 'O' dan spasi setelahnya

                # Menulis entitas yang sudah dibersihkan ke file output
                writer.writerow([chapter_id, verse_id, entity])

# Proses file input dan hasilkan output yang sudah dibersihkan
clean_entities(input_file_path, output_file_path)

# Konfirmasi bahwa file output telah disimpan
print(f"File hasil pembersihan disimpan di: {output_file_path}")


File hasil pembersihan disimpan di: /content/sample_data/output/output_cleaned_gpt4.tsv


## Membandingkan File DEV VAL TEST dengan Ekstraksi GPT-4

In [ ]:
import pandas as pd

# Membaca file TSV output_cleaned
output_cleaned = pd.read_csv('/content/sample_data/output/output_cleaned_gpt4.tsv', sep='\t')

# Membaca file unique_entities.txt
with open('/content/sample_data/output/output_text_cleaned.txt', 'r') as file:
    unique_entities = set(file.read().splitlines())  # Membaca semua entitas unik dan menjadikannya set

# Membandingkan entitas yang ada di output_cleaned dengan yang ada di unique_entities
# Filter entitas yang tidak ada di unique_entities
difference = output_cleaned[~output_cleaned['entitas'].isin(unique_entities)]

# Menyimpan hasil dalam format TSV
difference[['entitas', 'chapter_id', 'verse_id']].to_csv('difference_output_gpt4.tsv', sep='\t', index=False)

# Menampilkan output
print(difference[['entitas', 'chapter_id', 'verse_id']])


                                       entitas  chapter_id  verse_id
3                                   Pembalasan           1         4
34    Yang Maha Mengetahui lagi Maha Bijaksana           2        32
147                          Kerajaan Sulaiman           2       102
217      Yang Maha Perkasa lagi Maha Bijaksana           2       129
483                              tentara Talut           2       251
...                                        ...         ...       ...
7128                                   Hutamah         104         5
7131                       orang-orang Quraisy         106         1
7138                                 Abu Lahab         111         1
7141                             Yang Maha Esa         112         1
7144                                     subuh         113         1

[640 rows x 3 columns]


### Mengganti Ke Unicode

In [ ]:
import csv

# Path ke file input dan output
input_file_path = '/content/sample_data/output_cleaned.tsv'  # Ganti dengan file input Anda
output_file_path = '/content/sample_data/output_unicode.tsv'  # Ganti dengan file output yang ingin Anda simpan

# Fungsi untuk mengonversi karakter non-ASCII menjadi Unicode escape sequences
def convert_to_unicode_escape(text):
    # Mengonversi karakter non-ASCII menjadi escape sequences unicode
    return ''.join([f'\\u{ord(c):04x}' if ord(c) > 127 else c for c in text])

# Membaca file TSV dan mengonversi karakter-karakter non-ASCII
def process_tsv(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        reader = csv.reader(infile, delimiter='\t')

        # Menulis hasil ke file output
        with open(output_file, 'w', encoding='utf-8', newline='') as outfile:
            writer = csv.writer(outfile, delimiter='\t')

            header = next(reader)  # Membaca header
            writer.writerow(header)  # Menulis header ke file output

            # Proses setiap baris dan konversi entitas menjadi Unicode escape sequences
            for row in reader:
                chapter_id = row[1]
                verse_id = row[2]
                entity = row[0]

                # Konversi entitas ke Unicode escape
                entity_unicode = convert_to_unicode_escape(entity)

                # Menulis baris dengan entitas yang sudah dikonversi
                writer.writerow([entity_unicode, chapter_id, verse_id])

# Proses file input dan simpan hasilnya ke file output
process_tsv(input_file_path, output_file_path)

print(f"File hasil konversi disimpan di: {output_file_path}")


File hasil konversi disimpan di: /content/output_unicode.tsv
